In [ ]:
# import requests
# import pandas as pd
# import math

# base_url = "https://apis.data.go.kr/1741000/StanReginCd/getStanReginCdList"

# service_key = "340655d72cca6098386a1b96b9661ba12d173caaf583ad4fdda4d52d34b8daf7"
# region_name = "경상북도"
# rows_per_page = 1000

# url = f"{base_url}?ServiceKey={service_key}&pageNo=1&numOfRows={rows_per_page}&type=json&locatadd_nm={region_name}"

# response = requests.get(url)
# data = response.json()

# total_count = int(data["StanReginCd"][0]["head"][0]["totalCount"])
# total_pages = math.ceil(total_count / rows_per_page)

# merged_df = pd.DataFrame()

# for page_no in range(1, total_pages + 1):
#     url = f"{base_url}?ServiceKey={service_key}&pageNo={page_no}&numOfRows={rows_per_page}&type=json&locatadd_nm={region_name}"
#     response = requests.get(url)
#     items = response.json()["StanReginCd"][1]["row"]

#     page_df = pd.DataFrame(items)
#     new_df = page_df.loc[page_df['umd_cd'] == '000']
#     merged_df = pd.concat([merged_df, new_df])


# merged_df.to_csv("./data/행정표준코드.csv", index=False, encoding="utf-8-sig")


In [ ]:
import math

import pandas as pd
import requests

base_url = "https://api.odcloud.kr/api/15136368/v1/uddi:bbc26865-3049-42f5-a4b4-92c7f123a777?page=1&perPage=1000&returnType=json&serviceKey=340655d72cca6098386a1b96b9661ba12d173caaf583ad4fdda4d52d34b8daf7"


rows_per_page = 1000

response = requests.get(base_url)

total_count = int(response.json()["totalCount"])
total_pages = math.ceil(total_count / rows_per_page)

dfs = []

for page_no in range(1, 200):
    base_url = f"https://api.odcloud.kr/api/15136368/v1/uddi:bbc26865-3049-42f5-a4b4-92c7f123a777?page={page_no}&perPage=1000&returnType=json&serviceKey=340655d72cca6098386a1b96b9661ba12d173caaf583ad4fdda4d52d34b8daf7"
    response = requests.get(base_url)

    data = response.json()["data"]
    df = pd.DataFrame.from_records(data)
    code = df["행정구역코드"].astype("string")  # pandas StringDtype

    filtered = df.loc[(code.str.len().eq(5)) & (df["개정일자"] == "2025-04-01")]

    if filtered.shape[0] > 0:
        dfs.append(filtered)

    if page_no % 100 == 0:
        print(f"현재 페이지 : {page_no}")

merged_df = pd.concat(dfs, ignore_index=True)
merged_df = merged_df.sort_values(by=["시도명", "시군구명"])
merged_df.to_csv("./data/행정구역코드.csv")